In [70]:
# https://api.mongodb.com/python/current/tutorial.html
import pprint
from pymongo import MongoClient
import pandas as pd
import requests as req 
import json
import math
import re
import folium
from folium import plugins
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ["key"]   

client = MongoClient('mongodb://localhost:27017/')
db = client.ejercicio

In [2]:
# {$and:[{deadpooled_year:null}, {number_of_employees:{$ne:null}}, {"offices.latitude":{$ne:null}}, {"offices.latitude":{$exists:true}}]}

In [3]:
empresas= db.companhias.find({"$and":[{"deadpooled_year":None}, {"number_of_employees":{"$ne":None}},{"total_money_raised":{"$ne":None}}, {"offices.latitude":{"$ne":None}}, {"offices.latitude":{"$exists":"true"}}]})

In [4]:
df=pd.DataFrame(empresas)[["name","category_code","number_of_employees","founded_year", "total_money_raised","offices"]]

In [5]:
df.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,Digg,news,60,2004.0,$45M,"[{'description': None, 'address1': '135 Missis..."
1,Geni,web,18,2006.0,$16.5M,"[{'description': 'Headquarters', 'address1': '..."
2,Fox Interactive Media,web,0,1979.0,$0,"[{'description': '', 'address1': '407 N Maple ..."
3,Scribd,news,50,2007.0,$25.8M,"[{'description': 'HQ', 'address1': '539 Bryant..."
4,MeetMoi,social,15,2007.0,$5.58M,"[{'description': None, 'address1': '', 'addres..."


In [6]:
df1=df.copy()

In [7]:
res = req.get("https://api.exchangeratesapi.io/latest?base=USD").json()

In [8]:
def cambiar(string):
    if string.startswith("$"):
        string=string.replace("$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= float(string)*1000000
        elif string.endswith("k"):
            string=string.replace("k","")
            string= float(string)*1000
        elif string.endswith("B"):
            string=string.replace("B","")
            string= float(string)*1000000000
        else:
            string=float(string)
    
    elif string.startswith("€"):
        string=string.replace("€","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["EUR"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["EUR"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["EUR"]
        else:
            string=float(string)
            
    elif string.startswith("£"):
        string=string.replace("£","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["GBP"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["GBP"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["GBP"]
        else:
            string=float(string)

    elif string.startswith("¥"):
        string=string.replace("¥","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["JPY"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["JPY"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["JPY"]
        else:
            string=float(string)
      
    elif string.startswith("C$"):
        string=string.replace("C$","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["CAD"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["CAD"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["CAD"]
        else:
            string=float(string)
            
    elif string.startswith("kr"):
        string=string.replace("kr","")
        if string.endswith("M"):
            string=string.replace("M","")
            string= (float(string)*1000000)/res["rates"]["SEK"]
        elif string.endswith("k"):
            string=string.replace("k","")
            string= (float(string)*1000)/res["rates"]["SEK"]
        elif string.endswith("B"):
            string=string.replace("B","")
            string= (float(string)*1000000000)/res["rates"]["SEK"]
        else:
            string=float(string)
        
    return string

In [9]:
lst=[]
for a in range(len(df1)):
    lst.append(df1["offices"][a][0]["city"])
df1["ciudad"]=lst

In [12]:
df1["total_money_raised"]=df1["total_money_raised"].apply(cambiar)

In [13]:
df1.head()

,name,category_code,number_of_employees,founded_year,total_money_raised,offices,ciudad
0,Digg,news,60,2004.0,45000000.0,"[{'description': None, 'address1': '135 Missis...",San Francisco
1,Geni,web,18,2006.0,16500000.0,"[{'description': 'Headquarters', 'address1': '...",West Hollywood
2,Fox Interactive Media,web,0,1979.0,0.0,"[{'description': '', 'address1': '407 N Maple ...",Beverly Hills
3,Scribd,news,50,2007.0,25800000.0,"[{'description': 'HQ', 'address1': '539 Bryant...",San Francisco
4,MeetMoi,social,15,2007.0,5580000.0,"[{'description': None, 'address1': '', 'addres...",New York City


In [14]:
def getFirst(data):
    data = data['offices']
#    return (len(data),data[0]['latitude'],data[0]['longitude'])

    # Only create the geoJSON object if all geodata is available
    principal = None
    if data[0]['latitude'] and data[0]['longitude']:
        principal = {
            "type":"Point",
            "coordinates":[data[0]['longitude'], data[0]['latitude']]
        }

    return {
        "totalOffices": len(data),
        "lat": data[0]['latitude'],
        "lng": data[0]['longitude'],
        "oficina_principal": principal
    }


first_office = df1[["offices"]].apply(getFirst, result_type="expand", axis=1)

In [15]:
first_office.head()

,lat,lng,oficina_principal,totalOffices
0,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,...",1
1,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,...",1
2,34.076179,-118.394170,"{'type': 'Point', 'coordinates': [-118.39417, ...",1
3,37.789634,-122.404052,"{'type': 'Point', 'coordinates': [-122.404052,...",1
4,40.757929,-73.985506,"{'type': 'Point', 'coordinates': [-73.985506, ...",1


In [16]:
df_clean = pd.concat([df1,first_office], axis=1)[["name","lng","lat", "oficina_principal","totalOffices", "category_code", "number_of_employees", "founded_year", "total_money_raised", "ciudad"]]

In [17]:
df_clean.head()

,name,lng,lat,oficina_principal,totalOffices,category_code,number_of_employees,founded_year,total_money_raised,ciudad
0,Digg,-122.394523,37.764726,"{'type': 'Point', 'coordinates': [-122.394523,...",1,news,60,2004.0,45000000.0,San Francisco
1,Geni,-118.393064,34.090368,"{'type': 'Point', 'coordinates': [-118.393064,...",1,web,18,2006.0,16500000.0,West Hollywood
2,Fox Interactive Media,-118.394170,34.076179,"{'type': 'Point', 'coordinates': [-118.39417, ...",1,web,0,1979.0,0.0,Beverly Hills
3,Scribd,-122.404052,37.789634,"{'type': 'Point', 'coordinates': [-122.404052,...",1,news,50,2007.0,25800000.0,San Francisco
4,MeetMoi,-73.985506,40.757929,"{'type': 'Point', 'coordinates': [-73.985506, ...",1,social,15,2007.0,5580000.0,New York City


In [18]:
df_clean.to_json('limpio.json', orient="records")

In [19]:
df_clean.to_csv('limpio.csv')

In [20]:
oficinas=df_clean.copy()

In [69]:
oficinas=oficinas[(oficinas["total_money_raised"]>1000000) & (oficinas["founded_year"]>2005)]
lista_paises=list(oficinas.ciudad.value_counts().head(20).index)
print(lista_paises)
df20=oficinas[oficinas["ciudad"].isin(lista_paises)]
df20.ciudad.value_counts()

['San Francisco', 'New York', 'Seattle', 'London', 'Mountain View', 'San Mateo', 'Palo Alto', 'Los Angeles', 'San Jose', 'Boulder', 'Paris', 'Chicago', 'Portland', 'Boston', 'Berlin', 'Cambridge', 'Austin', 'San Diego', 'Sunnyvale', 'Redwood City']


San Francisco    83
New York         45
Seattle          19
London           19
Mountain View    15
San Mateo        14
Palo Alto        14
Boulder           9
Los Angeles       9
San Jose          9
Chicago           7
Paris             7
Boston            6
Portland          6
Cambridge         5
Berlin            5
San Diego         4
Redwood City      4
Austin            4
Sunnyvale         4
Name: ciudad, dtype: int64

In [67]:
df_clean.to_json('mirar.json', orient="records")

In [55]:
def calcMedia(c):
    bd = oficinas[oficinas["ciudad"]==c]
    contador=bd.ciudad.count()
    suma=bd.total_money_raised.sum()
    media=round(suma/contador)
    return media
res=[]
for c in lista_paises:
    a=calcMedia(c)
    res.append(a)
print(res)

[41958871.0, 17979663.0, 7014211.0, 6239012.0, 38832833.0, 23011429.0, 24362138.0, 22922222.0, 13466667.0, 20353333.0, 36412528.0, 4385714.0, 7950000.0, 24351667.0, 11989170.0, 29200000.0, 9225000.0, 8875000.0, 4812500.0, 13570000.0]


In [37]:
ciudad_media=pd.DataFrame()
ciudad_media["ciudad"]=lista_paises
ciudad_media["media"]=res

In [64]:
ciudad_media=ciudad_media.sort_values(["media"], ascending=False)
ciudad_media

,ciudad,media
0,San Francisco,41958871.0
4,Mountain View,38832833.0
10,Paris,36412528.0
15,Cambridge,29200000.0
6,Palo Alto,24362138.0
13,Boston,24351667.0
5,San Mateo,23011429.0
7,Los Angeles,22922222.0
9,Boulder,20353333.0
1,New York,17979663.0


In [75]:
ciudas=ciudad_media.ciudad[0]

lonlat= req.get("https://api.opencagedata.com/geocode/v1/json?q={}&key={}".format(ciudas, api_key)).json()
lonlat
llat=lonlat["results"][0]["geometry"]["lat"]
llng=lonlat["results"][0]["geometry"]["lng"]
print(llat,llng)



37.7790262 -122.4199061


In [65]:
m = folium.Map([37, -72], zoom_start=2)
stationArr = df20[['lat', 'lng']].values
m.add_child(plugins.HeatMap(stationArr, radius=15))
m

In [ ]:
#https://docs.mongodb.com/manual/reference/operator/query/near/index.html
def findNear(geopoint, radio_max_meters=2000):
    return db.empresas.find({"oficina_principal": {"$near": {"$geometry": geopoint,"$maxDistance": radio_max_meters,}}})

park_avenue_con_21_street = {
    "type":"Point",
    "coordinates":[-73.987308,40.738935]
}
radio_max_meters = 10000
num_offices = findNear(park_avenue_con_21_street, radio_max_meters).count()

print(f"Hay {num_offices} oficinas cerca a {radio_max_meters} metros")
print("La mas cercana es")
print(list(findNear(park_avenue_con_21_street, radio_max_meters).limit(1)))